In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import pandas_profiling

import numpy as np

import seaborn as sns
sns.set()

import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("../../data/Sales_Multiseries_training.csv")

In [4]:
# lets bring in the the week when the sale happened
data["date_"] = pd.to_datetime(arg=data.Date)
sav_data = data[data.Store == "Savannah"]
len(sav_data)

714

In [5]:
# lets bring in the the week when the sale happened
min(sav_data["date_"])

Timestamp('2012-07-01 00:00:00')

In [6]:
max(sav_data["date_"])

Timestamp('2014-06-14 00:00:00')

In [7]:
data = [go.Scatter(x=sav_data["date_"], y=sav_data["Sales"], line = dict(color = '#17BECF'))]

layout = dict(
    title = "Sales - Savannah",
    xaxis = dict(
        range = ['2012-07-01','2014-06-14'])
)

fig = dict(data=data, layout=layout)

py.iplot(data, filename = 'time-series-simple')

In [8]:
# 600 rows to train # 214 rows to test
data_to_process = sav_data.iloc[:,2:3].values
train = data_to_process[:600]
test = data_to_process[601:]

In [10]:
train.head(10)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [10]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

training_set_scaled = sc.fit_transform(train)

In [11]:
# Creating a data structure with 7 timesteps and t+1 output

X_train = []
y_train = []
for i in range(7, 600):
    X_train.append(training_set_scaled[i-7:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [12]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [14]:
regressor = Sequential()
regressor.add(LSTM(units = 5, input_shape = (None, 1)))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
history = regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
593/593 [==============================] - 5s 8ms/step - loss: 0.0859
Epoch 2/100
593/593 [==============================] - 0s 322us/step - loss: 0.0507
Epoch 3/100
593/593 [==============================] - 0s 322us/step - loss: 0.0267
Epoch 4/100
593/593 [==============================] - 0s 300us/step - loss: 0.0131
Epoch 5/100
593/593 [==============================] - 0s 302us/step - loss: 0.0083
Epoch 6/100
593/593 [==============================] - 0s 310us/step - loss: 0.0079
Epoch 7/100
593/593 [==============================] - 0s 302us/step - loss: 0.0078
Epoch 8/100
593/593 [==============================] - 0s 300us/step - loss: 0.0078
Epoch 9/100
593/593 [==============================] - 0s 324us/step - loss: 0.0077
Epoch 10/100
593/593 [==============================] - 0s 304us/step - loss: 0.0077
Epoch 11/100
593/593 [==============================] - 0s 292us/step - loss: 0.0076
Epoch 12/100
593/593 [==============================] - 0s 315us/step - loss

In [15]:
test = sc.fit_transform(test)
test_full = np.concatenate((train[0:600], test), axis = 0)

In [16]:
inputs = []
for i in range(601, 714):
    inputs.append(test_full[i-7:i, 0])
    
inputs = np.array(inputs)
inputs = np.reshape(inputs, (inputs.shape[0], inputs.shape[1], 1))
predicted = regressor.predict(inputs)
predicted_price = sc.inverse_transform(predicted)

In [17]:
max(sav_data["date_"][600:])

Timestamp('2014-06-14 00:00:00')

In [18]:
data_actual = go.Scatter(x=sav_data["date_"][601:], 
                         y=test_full[600:], 
                         line = dict(color = '#0B6623'),
                         name="Actual")
data_pred = go.Scatter(x=sav_data["date_"][601:], 
                       y=predicted_price, 
                       line = dict(color = '#B22222'), 
                       name = "Predicted")

layout = dict(
    title = "Sales - Savannah",
    xaxis = dict(
        range = ['2014-02-21','2014-06-14'])
)

data_final = [data_actual, data_pred]

fig = dict(data=data_final, layout=layout)
py.iplot(fig, filename = 'Actual vs Predicted')

In [19]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(test_full[600:], predicted_price)

In [20]:
rmse

7059580421.461546